# Gramener Case Study


In [ ]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#Importing the dataset
df = pd.read_csv('loan.csv')
df.head()

In [ ]:
print(df.shape,'\n')
print(df.info())

## Data cleaning

In [ ]:
# checking for missing values
round(df.isnull().mean() * 100,2)

### we can see columns even have 100 % missing values

##### removing the columns having more than 90% missing values

In [ ]:
# printing columns having more than 90% missing values
missing_columns = df.columns[round(df.isnull().mean() * 100,2) > 90]
missing_columns

In [ ]:
# dropping above columns
df = df.drop(missing_columns, axis=1)

In [ ]:
# updated shape of dataframe
print(df.shape)

### checking again for percentage of missing values after dropping columns greater than 90% missing values

In [ ]:
round(df.isnull().mean() * 100,2)

##### Still  two columns i.e. 'desc' and 'mths_since_last_delinq ' now remain that have high percentage of missing values
###### lets check them

In [ ]:
# checking what desc and mths_since_last_delinq contain
df.loc[:, ['desc', 'mths_since_last_delinq']].head(10)

since 'desc' is a column of type text and 'mths_since_last_delinq' data is unknown at the time of loan application, it cannot be used as a predictor of default at the time of loan approval.

###### Therefore we drop these two columns as well

In [ ]:
df = df.drop(['desc', 'mths_since_last_delinq'], axis=1)

### analysing missing values in rows

In [ ]:
df.index[round(df.isnull().sum(axis=1))>5]

###### i.e. no rows have greater than 5 missing values

In [ ]:
# new shape after data cleani=sing
df.shape

In [ ]:
# checking data type of remaining colunmns
df.info()

In [ ]:
# checking what int_rate and emp_length contain
df.loc[:, ['int_rate','emp_length']].head(10)

In [ ]:
# The column int_rate is object type,
#converting to float
df['int_rate'] = df['int_rate'].apply(lambda x: pd.to_numeric(x.split("%")[0]))

In [ ]:
df.loc[:, ['int_rate','emp_length']].head(10)

In [ ]:
# checking for null values
round(df['emp_length'].isnull().mean() * 100,2)

In [ ]:
# removing missing values from column 'emp_length'
df = df[~df['emp_length'].isnull()]

A regular expression in a programming language is a special text string used for describing a search pattern.
using regular expression to extract numeric values from string

In [ ]:
import re
df['emp_length'] = df['emp_length'].apply(lambda x : re.findall('\d+',str(x))[0])

In [ ]:
# The column emp_length is object type, converting it to integer type
df['emp_length'] = df['emp_length'].apply(lambda x : pd.to_numeric(x))

In [ ]:
df.loc[:, ['int_rate','emp_length']].head(10)

#### Now, the customer behaviour variables are not available at the time of loan application, and thus they cannot be used as predictors for credit approval. So, removing those columns is a better idea

In [ ]:
non_pred = ["delinq_2yrs", "earliest_cr_line", "inq_last_6mths", "open_acc", "pub_rec", 
                 "revol_bal", "revol_util", "total_acc", "out_prncp", "out_prncp_inv", "total_pymnt", 
                 "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", 
                 "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt",
                 "last_credit_pull_d",  "application_type"]

In [ ]:
# removing behaviour variables
df = df.drop(non_pred,axis=1)

In [ ]:
# updated shape of dataframe
df.shape

In [ ]:
df.info()

In [ ]:
df.loc[:, ['title','purpose']].head(10)

columns title and purpose serve similar purpose so we can drop title column.
Also columns like zip_code, address, state, etc. are useless for prediction

In [ ]:
# dropping columns
df = df.drop(['title', 'url', 'zip_code', 'addr_state'],axis=1)

In [ ]:
# updated shape
df.shape

## Target variable: LOAN STATUS

#### The target variable, which we want to compare across the independent variables, is loan status. 
#### The strategy is to figure out compare the average default rates across various independent variables and identify the ones that affect default rate the most.

In [ ]:
# Assessing values associated with target variable
df.loan_status.unique()

In [ ]:
# count of various values associated with loan status
df.loan_status.value_counts()

In [ ]:
# removing 'current' values from column loan status
df = df[df['loan_status']!='Current']

#Labelling Fully paid as 0 and charged off as 1
df['loan_status'] = df['loan_status'].apply(lambda x: 0 if x=='Fully Paid' else 1)

#Converting to numeric type
df['loan_status'] = df['loan_status'].apply(lambda x: pd.to_numeric(x))

In [ ]:
#Summarising the values
df['loan_status'].value_counts()

# Univariate analysis

In [ ]:
#default percentage 
round(df['loan_status'].mean() * 100,2)

In [ ]:
# function for plotting categorical variable
def cat_var(var):
    sns.barplot(x=var, y='loan_status', data = df)
    plt.show()

In [ ]:
cat_var('grade')

from A to G, the default rate increases which indicates riskiness of the loan.

In [ ]:
cat_var('term')

60 months loan default rate is more than 36 months loan

In [ ]:
plt.figure(figsize=(18,4))
cat_var('sub_grade')

A1 is better than A2 better than A3 and so on 

In [ ]:
#purpose
plt.figure(figsize=(18,4))
cat_var('purpose')

small business loans defualt the most!

In [ ]:
#verification status
cat_var('verification_status')

Verified loan defaults are greater than the one which are not verified!!!

In [ ]:
#home-ownership
cat_var('home_ownership')

home_ownership is not of much importance

In [ ]:
df.loc[:, ['issue_d']].head(10)

In [ ]:
df['issue_d'].dtype

converting issue_d column into date and time format and then extracting year and month from it

In [ ]:
from datetime import datetime
df['issue_d'] = df['issue_d'].apply(lambda x: datetime.strptime(x, '%b-%y'))

In [ ]:
#extracting month and year from issue_date
df['month'] = df['issue_d'].apply(lambda x: x.month)
df['year'] = df['issue_d'].apply(lambda x: x.year)

In [ ]:
df['month'].value_counts()

In [ ]:
df['year'].value_counts()

 number of loans has increased steadily across years.

In [ ]:
#Default rate accross years
plt.figure(figsize=(12,4))
cat_var('year')

In [ ]:
#comparing defaults accross months
plt.figure(figsize=(14,4))
cat_var('month')

In [ ]:
# variation across continuous variables.
plt.figure(figsize=(14,4))
sns.distplot(df['loan_amnt'])
plt.show()

## Binning loan amount variable into small, medium, high and  very high discrete categories

In [ ]:
#binnig the loan amount
def binned(n):
    if n<5000:
        return 'low'
    elif n>5000 and n<15000:
        return 'medium'
    elif n>15000 and n<25000:
        return 'high'
    else:
        return 'very high'

df['loan_amnt'] = df['loan_amnt'].apply(lambda x: binned(x))   

In [ ]:
df['loan_amnt'].value_counts()

In [ ]:
# let's also convert funded amount invested to bins
df['funded_amnt_inv'] = df['funded_amnt_inv'].apply(lambda x: binned(x))

In [ ]:
plt.figure(figsize=(14,4))
cat_var('loan_amnt')

Higher the loan amount, higher the default rate

In [ ]:
# funded amount invested
plt.figure(figsize=(14,4))
cat_var('funded_amnt_inv')

In [ ]:
# Binning interest rate to low, medium, high
def rate(n):
    if n <= 10:
        return 'low'
    elif n > 10 and n <=15:
        return 'medium'
    else:
        return 'high'
    
    
df['int_rate'] = df['int_rate'].apply(lambda x: rate(x))

In [ ]:
# binnng debt to income ratio
def ratio(n):
    if n <= 10:
        return 'low'
    elif n > 10 and n <=20:
        return 'medium'
    else:
        return 'high'
    

df['dti'] = df['dti'].apply(lambda x: ratio(x))

In [ ]:
# funded amount
def funded_amount(n):
    if n <= 5000:
        return 'low'
    elif n > 5000 and n <=15000:
        return 'medium'
    else:
        return 'high'
    
df['funded_amnt'] = df['funded_amnt'].apply(lambda x: funded_amount(x))

In [ ]:
# annual income
def annual_income(n):
    if n <= 50000:
        return 'low'
    elif n > 50000 and n <=100000:
        return 'medium'
    elif n > 100000 and n <=150000:
        return 'high'
    else:
        return 'very high'

df['annual_inc'] = df['annual_inc'].apply(lambda x: annual_income(x))

In [ ]:
#checking for null values
df['emp_length'].isnull().sum()

In [ ]:
# binning emp_length
def emp_length(n):
    if n <= 1:
        return 'fresher'
    elif n > 1 and n <=3:
        return 'junior'
    elif n > 3 and n <=7:
        return 'senior'
    else:
        return 'expert'

df['emp_length'] = df['emp_length'].apply(lambda x: emp_length(x))

In [ ]:
# installment
def installment(n):
    if n <= 200:
        return 'low'
    elif n > 200 and n <=400:
        return 'medium'
    elif n > 400 and n <=600:
        return 'high'
    else:
        return 'very high'
    
df['installment'] = df['installment'].apply(lambda x: installment(x))

In [ ]:
plt.figure(figsize=(12,4))
cat_var('int_rate')

High interest rates default more!

In [ ]:
plt.figure(figsize=(12,4))
cat_var('dti')

In [ ]:
plt.figure(figsize=(12,4))
cat_var('funded_amnt')

In [ ]:
plt.figure(figsize=(12,4))
cat_var('dti')

In [ ]:
plt.figure(figsize=(12,4))
cat_var('installment')

In [ ]:
plt.figure(figsize=(12,4))
cat_var('annual_inc')

Lower the annual income, higher the default rate

In [ ]:
plt.figure(figsize=(10,4))
cat_var('emp_length')

emp_length is not a good predictor of default

# SEGMENTED UNIVARIATE ANALYSIS

#### some of the important predictors are purpose of the loan, interest rate, annual income, grade etc.

In [ ]:
# purpose: 
plt.figure(figsize=(18, 8))
cat_var('purpose')

small business loans defualt the most, then renewable energy and education

In [ ]:
# number of loans for each type (purpose) of the loan
plt.figure(figsize=(18, 8))
sns.countplot(x='purpose', data=df)
plt.show()

#### top 4 types of loans based on purpose: consolidation, credit card, home improvement and major purchase.

In [ ]:
main_purp = ["credit_card","debt_consolidation","home_improvement","major_purchase"]
df = df[df['purpose'].isin(main_purp)]
df['purpose'].value_counts()

In [ ]:
# plotting number of loans by purpose
plt.figure(figsize=(14, 6))
sns.countplot(x=df['purpose'])
plt.show()

In [ ]:
# function which takes a categorical variable and plots the default rate segmented by purpose 

def seg(cat_var):
    plt.figure(figsize=(18, 6))
    sns.barplot(x=cat_var, y='loan_status', hue='purpose', data=df)
    plt.show()

In [ ]:
# comparing the default rates across two categorical variables i.e.
# purpose of loan (constant) and another categorical variable (which changes)
seg('term')

In [ ]:
seg('home_ownership')

In [ ]:
seg('grade')

In [ ]:
seg('year')

In [ ]:
seg('emp_length')

In [ ]:
seg('loan_amnt')

In [ ]:
seg('int_rate')

In [ ]:
seg('annual_inc')

In [ ]:
seg('installment')

In [ ]:
seg('dti')

### checking variation of the default rate across the categories

In [ ]:
# function takes in a categorical variable and computes the mean  default rate across the categories
def diff(cat_var):
    default_rates = df.groupby(cat_var).loan_status.mean().sort_values(ascending=False)
    return (round(default_rates, 2), round(default_rates[0] - default_rates[-1], 2))

default_rates, diff = diff('annual_inc')
print(default_rates) 
print(diff)

### Thus, there is a 6% increase in default rate as you go from high to low annual income.